<a href="https://colab.research.google.com/github/Dev-Satish-02/AmazonScraper/blob/main/AmazonScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4 requests pandas

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [ ]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=arduino"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [ ]:
amazon_df

,title,price,rating,reviews,availability
3,Arduino UNO R4 WiFi [ABX00087] – Renesas RA4M1...,,4.5 out of 5 stars,"1,341 ratings",In Stock
4,UNO R3 SMD Board compatible with Arduino Devel...,,3.8 out of 5 stars,214 ratings,Not Available
5,Improved Version Nano V3 Board Compatible with...,,5.0 out of 5 stars,,Not Available
7,vbsdfb19594,,4.4 out of 5 stars,161 ratings,Not Available
9,Arduino Uno REV3 [A000066] – ATmega328P Microc...,,4.5 out of 5 stars,"9,829 ratings",In Stock
10,Arduino Nano [A000005] – Compact ATmega328P Mi...,,4.6 out of 5 stars,"1,074 ratings",In Stock
11,"Arduino Mega 2560 REV3 [A000067] – ATmega2560,...",,4.8 out of 5 stars,"1,751 ratings",In Stock
12,Nano R3 CH340 Development Board with ATmega328...,,4.6 out of 5 stars,13 ratings,Not Available
17,UNO R3 DIP Board ATmega328P ATmega16U2 Compati...,,5.0 out of 5 stars,,Not Available
23,Arduino Due with Headers [A000062] – 32-bit AR...,,4.5 out of 5 stars,575 ratings,In Stock
